## C-Stress - Computing Stress Episodes from ECG Data




**Reference:** [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4631393/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4631393/)

cStress analyzes a minute’s worth of ECG and respiration data, and, if this minute is not confounded by physical activity, outputs probability of stress. It is trained using data collected from 21 participants who were subjected to three validated stressors — public speaking, mental arithmetic, and cold-pressor. The ground-truth in the lab is collected for each minute, based on knowledge of starts and ends of simulated stressors. This enabled us to create a fine-grained model of physiological stress activation (at one minute resolution). The model is evaluated also at the same fine-grained level, i.e. once a minute on the lab data. In the field, self-reported stress in response to Ecological Momentary Assessment (EMA) prompts are used as ground-truths. We note that even though cStress produces a stress value for each minute, participants were prompted for self-report of stress only 15 times a day (in order to limit participant fatigue). Field validation of cStress is, therefore, limited to these self-reports.

This example applies the cStress model to a publically available dataset, Wearable Stress and Affect Detection ([WESAD](https://archive.ics.uci.edu/ml/datasets/WESAD+%28Wearable+Stress+and+Affect+Detection%29)).  WESAD is a publicly available dataset for wearable stress and affect detection. This multimodal dataset features physiological and motion data, recorded from both a wrist- and a chest-worn device, of 15 subjects during a lab study. The following sensor modalities are included: blood volume pulse, electrocardiogram, electrodermal activity, electromyogram, respiration, body temperature, and three-axis acceleration. Moreover, the dataset bridges the gap between previous lab studies on stress and emotions, by containing three different affective states (neutral, stress, amusement). In addition, self-reports of the subjects, which were obtained using several established questionnaires, are contained in the dataset.

**WESAD Reference:** Philip Schmidt, Attila Reiss, Robert Duerichen, Claus Marberger and Kristof Van Laerhoven. 2018. Introducing WESAD, a multimodal dataset for Wearable Stress and Affect Detection. In 2018 International Conference on Multimodal Interaction (ICMI 2018), October 16-20, 2018, Boulder, CO, USA. ACM, New York, NY, USA, 9 pages. [https://doi.org/10.1145/3242969.3242985](https://doi.org/10.1145/3242969.3242985)

## Setting Up Environment


Colab does not contain the necessary runtime enviornments necessary to run Cerebral Cortex.  The following commands will download and install these tools, framework, and datasets.

### Download Sample WESAD ECG Dataset
Set `ALL_USERS=true` if you want to process all participants data. Setting `ALL_USERS=false` will only process one participant data.  

* One participant's Data [http://mhealth.md2k.org/images/datasets/s2_data.tar.bz2](http://mhealth.md2k.org/images/datasets/s2_data.tar.bz2)
  * Processing time ~10-14 minutes
* All participants' data [http://mhealth.md2k.org/images/datasets/cc_data.tar.bz2](http://mhealth.md2k.org/images/datasets/cc_data.tar.bz2)
  * Processing time ~60-75 minutes


In [ ]:
ALL_USERS=False

In [ ]:
import importlib, sys, os
from os.path import expanduser
sys.path.insert(0, os.path.abspath('..'))

DOWNLOAD_USER_DATA=True
#ALL_USERS=False #this will only  work if DOWNLOAD_USER_DATA=True
IN_COLAB = 'google.colab' in sys.modules
MD2K_JUPYTER_NOTEBOOK = "MD2K_JUPYTER_NOTEBOOK" in os.environ
if (get_ipython().__class__.__name__=="ZMQInteractiveShell"): IN_JUPYTER_NOTEBOOK = True
JAVA_HOME_DEFINED = "JAVA_HOME" in os.environ
SPARK_HOME_DEFINED = "SPARK_HOME" in os.environ
PYSPARK_PYTHON_DEFINED = "PYSPARK_PYTHON" in os.environ
PYSPARK_DRIVER_PYTHON_DEFINED = "PYSPARK_DRIVER_PYTHON" in os.environ
HAVE_CEREBRALCORTEX_KERNEL = importlib.util.find_spec("cerebralcortex") is not None
SPARK_VERSION = "3.1.2"
SPARK_URL = "https://archive.apache.org/dist/spark/spark-"+SPARK_VERSION+"/spark-"+SPARK_VERSION+"-bin-hadoop2.7.tgz"
SPARK_FILE_NAME = "spark-"+SPARK_VERSION+"-bin-hadoop2.7.tgz"
CEREBRALCORTEX_KERNEL_VERSION = "3.3.14"

DATA_PATH = expanduser("~")
if DATA_PATH[:-1]!="/":
    DATA_PATH+="/"
USER_DATA_PATH = DATA_PATH+"cc_data/"

if MD2K_JUPYTER_NOTEBOOK:
    print("Java, Spark, and CerebralCortex-Kernel are installed and paths are already setup.")
else:

    SPARK_PATH = DATA_PATH+"spark-"+SPARK_VERSION+"-bin-hadoop2.7/"
    

    if(not HAVE_CEREBRALCORTEX_KERNEL):
        print("Installing CerebralCortex-Kernel")
        !pip -q install cerebralcortex-kernel==$CEREBRALCORTEX_KERNEL_VERSION
    else:
        print("CerebralCortex-Kernel is already installed.")

    if not JAVA_HOME_DEFINED:
        if not os.path.exists("/usr/lib/jvm/java-8-openjdk-amd64/") and not os.path.exists("/usr/lib/jvm/java-11-openjdk-amd64/"):
            print("\nInstalling/Configuring Java")
            !sudo apt update
            !sudo apt-get install -y openjdk-8-jdk-headless
            os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
        elif os.path.exists("/usr/lib/jvm/java-8-openjdk-amd64/"):
            print("\nSetting up Java path")
            os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
        elif  os.path.exists("/usr/lib/jvm/java-11-openjdk-amd64/"):
            print("\nSetting up Java path")
            os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
    else:
        print("JAVA is already installed.")

    if (IN_COLAB or IN_JUPYTER_NOTEBOOK) and not MD2K_JUPYTER_NOTEBOOK:
        if SPARK_HOME_DEFINED:
            print("SPARK is already installed.")
        elif not os.path.exists(SPARK_PATH):
            print("\nSetting up Apache Spark ", SPARK_VERSION)
            !pip -q install findspark
            import pyspark
            spark_installation_path = os.path.dirname(pyspark.__file__)
            import findspark
            findspark.init(spark_installation_path)
            if not os.getenv("PYSPARK_PYTHON"):
                os.environ["PYSPARK_PYTHON"] = os.popen('which python3').read().replace("\n","")
            if not os.getenv("PYSPARK_DRIVER_PYTHON"):
                os.environ["PYSPARK_DRIVER_PYTHON"] = os.popen('which python3').read().replace("\n","")
        else:
            print("SPARK is already installed.")
    else:
        raise SystemExit("Please check your environment configuration at: https://github.com/MD2Korg/CerebralCortex-Kernel/")

if DOWNLOAD_USER_DATA:
    if not os.path.exists(USER_DATA_PATH):
        if ALL_USERS:
            print("\nDownloading all users' data.")
            !rm -rf $USER_DATA_PATH
            !wget -q http://mhealth.md2k.org/images/datasets/cc_data.tar.bz2 && tar -xf cc_data.tar.bz2 -C $DATA_PATH && rm cc_data.tar.bz2
        else:
            print("\nDownloading a user's data.")
            !rm -rf $USER_DATA_PATH
            !wget -q http://mhealth.md2k.org/images/datasets/s2_data.tar.bz2 && tar -xf s2_data.tar.bz2 -C $DATA_PATH && rm s2_data.tar.bz2
    else:
        print("Data already exist. Please remove folder", USER_DATA_PATH, "if you want to download the data again")

Installing CerebralCortex-Kernel
     |████████████████████████████████| 194 kB 39.4 MB/s 
     |████████████████████████████████| 105 kB 52.3 MB/s 
     |████████████████████████████████| 100 kB 7.8 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 721 kB 58.8 MB/s 
     |████████████████████████████████| 20.6 MB 1.3 MB/s 
     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 44 kB 2.4 MB/s 
     |████████████████████████████████| 636 kB 40.3 MB/s 
     |████████████████████████████████| 1.3 MB 28.1 MB/s 
     |████████████████████████████████| 21.8 MB 1.4 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 198 kB 60.5 MB/s 
     |████████████████████████████████| 554 kB 38.2 MB/s 

Setting up Java path

Setting up Apache Spark  3.1.2



## C-Stress Execution Pipeline

### Create CerebralCortex-Kernel Object and Load Data

In [ ]:
from cerebralcortex.kernel import Kernel

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
CC = Kernel(cc_configs="default", study_name="wesad")

In [ ]:
CC.list_streams()

['wesad.chest.ecg',
 'wesad.chest.resp',
 'wesad.chest.temp',
 'wesad.chest.eda',
 'wesad.wrist.acc',
 'wesad.wrist.temp',
 'wesad.quest',
 'wesad.wrist.bvp',
 'wesad.label',
 'wesad.wrist.eda',
 'wesad.chest.emg',
 'wesad.chest.acc']

In [ ]:
ecg_data = CC.get_stream("wesad.chest.ecg")

In [ ]:
ecg_data.show(5, truncate=False)

+--------------------------+--------------------------+------------------+----+-------+
|timestamp                 |localtime                 |ecg               |user|version|
+--------------------------+--------------------------+------------------+----+-------+
|2017-05-22 02:15:25       |2017-05-22 02:15:25       |0.02142333984375  |s2  |1      |
|2017-05-22 02:15:25.001429|2017-05-22 02:15:25.001429|0.02032470703125  |s2  |1      |
|2017-05-22 02:15:25.002857|2017-05-22 02:15:25.002857|0.0165252685546875|s2  |1      |
|2017-05-22 02:15:25.004286|2017-05-22 02:15:25.004286|0.0167083740234375|s2  |1      |
|2017-05-22 02:15:25.005714|2017-05-22 02:15:25.005714|0.0116729736328125|s2  |1      |
+--------------------------+--------------------------+------------------+----+-------+
only showing top 5 rows



### Import Stress from ECG Algorithm and Download Model File

In [ ]:
from cerebralcortex.algorithms.ecg.autosense_data_quality import ecg_autosense_data_quality
from cerebralcortex.algorithms.ecg.autosense_rr_interval import get_rr_interval
from cerebralcortex.algorithms.ecg.hrv_features import get_hrv_features
from cerebralcortex.algorithms.stress_prediction.ecg_stress import compute_stress_probability
from cerebralcortex.algorithms.stress_prediction.stress_episodes import compute_stress_episodes
from cerebralcortex.algorithms.stress_prediction.stress_imputation import forward_fill_data, impute_stress_likelihood
from cerebralcortex.algorithms.utils.feature_normalization import normalize_features

**Parameters**

In [ ]:
sensor_name="respiban"
Fs=700
model_path=DATA_PATH+"stress_ecg_final.p"

**Download Stress Model File**

In [ ]:
!wget -q https://github.com/MD2Korg/CerebralCortex-Kernel/blob/master/cerebralcortex/markers/ecg_stress/model/stress_ecg_final.p?raw=true -O $model_path

### Compute data quality

In [ ]:
ecg_data_with_quality = ecg_autosense_data_quality(ecg_data,sensor_name=sensor_name,Fs=Fs).sort("timestamp")
ecg_data_with_quality.show(truncate=False)

+--------------------------+--------------------------+-------------------+----+-------+----------+
|timestamp                 |localtime                 |ecg                |user|version|quality   |
+--------------------------+--------------------------+-------------------+----+-------+----------+
|2017-05-22 02:15:25       |2017-05-22 02:15:25       |0.02142333984375   |s2  |1      |acceptable|
|2017-05-22 02:15:25.001429|2017-05-22 02:15:25.001429|0.02032470703125   |s2  |1      |acceptable|
|2017-05-22 02:15:25.002857|2017-05-22 02:15:25.002857|0.0165252685546875 |s2  |1      |acceptable|
|2017-05-22 02:15:25.004286|2017-05-22 02:15:25.004286|0.0167083740234375 |s2  |1      |acceptable|
|2017-05-22 02:15:25.005714|2017-05-22 02:15:25.005714|0.0116729736328125 |s2  |1      |acceptable|
|2017-05-22 02:15:25.007143|2017-05-22 02:15:25.007143|0.0048980712890625 |s2  |1      |acceptable|
|2017-05-22 02:15:25.008572|2017-05-22 02:15:25.008572|0.0027923583984375 |s2  |1      |acceptable|


### Compute RR intervals
The timing in milliseconds is computed from the ECG signals by measuring the time delta between successive R-peaks.

In [ ]:
ecg_rr = get_rr_interval(ecg_data_with_quality,Fs=Fs).sort("timestamp")
ecg_rr.show(truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/group_ops.py:84: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


+--------------------------+--------------------------+-------+----+---------+
|timestamp                 |localtime                 |version|user|rr       |
+--------------------------+--------------------------+-------+----+---------+
|2017-05-22 02:15:26.038595|2017-05-22 02:15:26.038595|1      |s2  |730.01685|
|2017-05-22 02:15:26.764326|2017-05-22 02:15:26.764326|1      |s2  |725.7312 |
|2017-05-22 02:15:27.534344|2017-05-22 02:15:27.534344|1      |s2  |770.0178 |
|2017-05-22 02:15:28.467222|2017-05-22 02:15:28.467222|1      |s2  |932.8779 |
|2017-05-22 02:15:29.330099|2017-05-22 02:15:29.330099|1      |s2  |862.8772 |
|2017-05-22 02:15:30.267263|2017-05-22 02:15:30.267263|1      |s2  |937.1638 |
|2017-05-22 02:15:31.137283|2017-05-22 02:15:31.137283|1      |s2  |870.02026|
|2017-05-22 02:15:31.943016|2017-05-22 02:15:31.943016|1      |s2  |805.7329 |
|2017-05-22 02:15:32.744463|2017-05-22 02:15:32.744463|1      |s2  |801.4468 |
|2017-05-22 02:15:33.527338|2017-05-22 02:15:33.5273

### Compute HRV features
The RR-intervals form the basis for computing the heart rate variability (HRV) features necessary for the cStress algorithmm.  Here, we are computing the following features for each 60 second window:

1. Variance
2. Inter Quartile Range
3. Very Low Frequency Power Spectrum (FREQ RANGE HERE)
4. Low Frequency Power Spectrum (FREQ RANGE HERE)
5. High Frequency Power Spectrum (FREQ RANGE HERE)
6. Low Frequency - High Frequency Power Specturm (???)
7. Mean
8. Median
9. 80th Percentile
10. 20th Percentile
11. Heartrate (60000/median)

The above features are stored as a vector contained in a single cell for ease of future computations.

In [ ]:
stress_features = get_hrv_features(ecg_rr).sort("timestamp")
stress_features.show(truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/group_ops.py:84: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


+--------------------------+------------------------------------------+--------------------------+----+-------+-----------------+-----------------+---------------------+--------------------+-------------------+--------------------+-----------------+----------------+-----------------+-----------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|timestamp                 |window                                    |localtime                 |user|version|var              |iqr              |vlf                  |lf                  |hf                 |lfhf                |mean             |median          |80th             |20th             |heartrate        |features                                                                                                                                           

### Normalize features

Standard z-score normalization is used.

In [ ]:
stress_features_normalized = normalize_features(stress_features,input_feature_array_name='features').sort("timestamp")
stress_features_normalized.show(truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/group_ops.py:84: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


+--------------------------+--------------------------+----+-------+-----------------+-----------------+---------------------+--------------------+-------------------+--------------------+-----------------+----------------+-----------------+-----------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------+
|timestamp                 |localtime                 |user|version|var              |iqr              |vlf                  |lf                  |hf                 |lfhf                |mean             |median          |80th     

### Compute stress probability

Use the pretrained model to evaluate the normalized features and produce a stress probability, which is the liklihood of stress.  Another way to look at this number is that the higher it is, the more severe the amount of physiological arrousal is and therefore the stress.  This results in a timeseries of stress probabilities that will now be turned into stress/non-stress episodes 

In [ ]:
ecg_stress_probability = compute_stress_probability(stress_features_normalized,model_path=model_path).sort("timestamp")
ecg_stress_probability.show(truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/group_ops.py:84: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


+--------------------------+------------------------------------------+--------------------------+-------------------+----+-------+
|timestamp                 |window                                    |localtime                 |stress_probability |user|version|
+--------------------------+------------------------------------------+--------------------------+-------------------+----+-------+
|2017-05-22 02:15:26.038595|{2017-05-22 02:15:00, 2017-05-22 02:16:00}|2017-05-22 02:15:26.038595|0.15747584780014517|s2  |1      |
|2017-05-22 02:16:00.570814|{2017-05-22 02:16:00, 2017-05-22 02:17:00}|2017-05-22 02:16:00.570814|0.972011643802874  |s2  |1      |
|2017-05-22 02:17:00.677904|{2017-05-22 02:17:00, 2017-05-22 02:18:00}|2017-05-22 02:17:00.677904|0.6565261686478017 |s2  |1      |
|2017-05-22 02:18:00.280697|{2017-05-22 02:18:00, 2017-05-22 02:19:00}|2017-05-22 02:18:00.280697|0.9076761749534125 |s2  |1      |
|2017-05-22 02:19:00.324928|{2017-05-22 02:19:00, 2017-05-22 02:20:00}|2017-

### Get Ground Truth Data

In [ ]:
labels = CC.get_stream("wesad.label")
labels.show(truncate=False)

+--------------------------+--------------------------+-----+----+-------+
|timestamp                 |localtime                 |label|user|version|
+--------------------------+--------------------------+-----+----+-------+
|2017-05-22 02:15:25       |2017-05-22 02:15:25       |0    |s2  |1      |
|2017-05-22 02:15:25.001429|2017-05-22 02:15:25.001429|0    |s2  |1      |
|2017-05-22 02:15:25.002857|2017-05-22 02:15:25.002857|0    |s2  |1      |
|2017-05-22 02:15:25.004286|2017-05-22 02:15:25.004286|0    |s2  |1      |
|2017-05-22 02:15:25.005714|2017-05-22 02:15:25.005714|0    |s2  |1      |
|2017-05-22 02:15:25.007143|2017-05-22 02:15:25.007143|0    |s2  |1      |
|2017-05-22 02:15:25.008572|2017-05-22 02:15:25.008572|0    |s2  |1      |
|2017-05-22 02:15:25.01    |2017-05-22 02:15:25.01    |0    |s2  |1      |
|2017-05-22 02:15:25.011429|2017-05-22 02:15:25.011429|0    |s2  |1      |
|2017-05-22 02:15:25.012857|2017-05-22 02:15:25.012857|0    |s2  |1      |
|2017-05-22 02:15:25.0142

### Join labels and stress probability data



In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from cerebralcortex.core.datatypes.datastream import DataStream

# window label data into 60 seconds chunks
win = F.window("timestamp", windowDuration="60 seconds",startTime='0 seconds')
windowed_labels=labels.groupby(["user","version",win]).agg(F.collect_list("label"))
# label each window
label_window_udf = udf(lambda s: max(set(s), key=s.count), IntegerType())
final_labels = DataStream(windowed_labels.withColumn("window_label", label_window_udf(F.col("collect_list(label)"))).drop("collect_list(label)"))
#join labels and stress probability data
stress_probability_labels = ecg_stress_probability.join(final_labels, ecg_stress_probability["window"]==final_labels["window"]).select(ecg_stress_probability["*"],final_labels["window_label"]).sort("window")

In [ ]:
stress_probability_labels.show(truncate=False)

+--------------------------+------------------------------------------+--------------------------+-------------------+----+-------+------------+
|timestamp                 |window                                    |localtime                 |stress_probability |user|version|window_label|
+--------------------------+------------------------------------------+--------------------------+-------------------+----+-------+------------+
|2017-05-22 02:15:26.038595|{2017-05-22 02:15:00, 2017-05-22 02:16:00}|2017-05-22 02:15:26.038595|0.15747584780014517|s2  |1      |0           |
|2017-05-22 02:16:00.570814|{2017-05-22 02:16:00, 2017-05-22 02:17:00}|2017-05-22 02:16:00.570814|0.972011643802874  |s2  |1      |0           |
|2017-05-22 02:17:00.677904|{2017-05-22 02:17:00, 2017-05-22 02:18:00}|2017-05-22 02:17:00.677904|0.6565261686478017 |s2  |1      |0           |
|2017-05-22 02:18:00.280697|{2017-05-22 02:18:00, 2017-05-22 02:19:00}|2017-05-22 02:18:00.280697|0.9076761749534125 |s2  |1      

In [ ]:
pdf = stress_probability_labels.toPandas()

### Visualize Stress Probabilities & Labels

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
user_ids = list(pdf.user.unique())
subplot_titles = ["Participant ID: {}".format(x.upper()) for x in list(pdf.user.unique())]

fig = make_subplots(rows=len(user_ids), cols=1, subplot_titles=subplot_titles)
row_id = 1

for sid in user_ids:
  if user_ids[-1]==sid:
    lagend = True
  else:
    lagend = False
  id = sid.replace("s","")
  fig.add_trace(go.Scatter(
    x=pdf.timestamp[pdf.user==sid],
    y=pdf.stress_probability[pdf.user==sid],
    showlegend=lagend,
    marker_color='black',
    name="Stress Probabilities"
  ), row=row_id, col=1)
  
  fig.add_trace(
    go.Scatter(
        x=pdf.timestamp[(pdf.window_label==1) & (pdf.user==sid)],
        y=[1]*len(pdf.window_label[(pdf.window_label==1) & (pdf.user==sid)]),
        showlegend=lagend,
        name="Baseline Episode",
        fill="tozeroy",
        fillcolor='rgba(0,0,255,0.3)',
        marker_color='rgba(0,0,255,0.3)',
        mode="lines"
    ), row=row_id, col=1)
  fig.add_trace(
    go.Scatter(
        x=pdf.timestamp[(pdf.window_label==2) & (pdf.user==sid)],
        y=[1]*len(pdf.window_label[(pdf.window_label==2) & (pdf.user==sid)]),
        showlegend=lagend,
        name="Stress Episode",
        fill="tozeroy",
        fillcolor='rgba(255,0,0,0.3)',
        marker_color='rgba(255,0,0,0.3)',
        mode="lines"
    ), row=row_id, col=1)
  fig.add_trace(
    go.Scatter(
        x=pdf.timestamp[(pdf.window_label==3) & (pdf.user==sid)],
        y=[1]*len(pdf.window_label[(pdf.window_label==3) & (pdf.user==sid)]),
        showlegend=lagend,
        name="Amusement Episode",
        fill="tozeroy",
        fillcolor='rgba(204,0,204,0.3)',
        marker_color='rgba(204,0,204,0.3)',
        mode="lines"
    ), row=row_id, col=1)
  fig.add_trace(
    go.Bar(
        x=pdf.timestamp[(pdf.window_label==4) & (pdf.user==sid)],
        y=[1]*len(pdf.window_label[(pdf.window_label==4) & (pdf.user==sid)]),
        showlegend=lagend,
        name="Meditation Episode",
        offset=0,
        width=60000*np.ones(len(pdf.window_label[(pdf.window_label==4) & (pdf.user==sid)])),
        marker_line_width=0,
        marker_color='rgba(23,0,56,0.3)',  
        marker_line_color='rgba(23,0,56,0.3)',      
    ), row=row_id, col=1)
  fig.update_xaxes(title_text="Timestamp", row=row_id, col=1)
  fig.update_yaxes(title_text="Stress Probability", row=row_id, col=1)
  row_id +=1
if len(user_ids)<2:
  height = 400
else:
  height = 3500
fig.update_layout(height=height, width=900, title_text="Stress Probabilities & Labels For Each Participant")
fig.show()

/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

